In [1]:
%cd yolov5

c:\freelancer\student ocr\yolov5


In [3]:
import cv2
import numpy as np
import pandas as pd
import numpy as np
net = cv2.dnn.readNetFromONNX("best.onnx")
file = open("coco.txt", "r")
classes = file.read().split('\n')
import os

# Directory path where your images are located
folder_path = "../df"

# Get a list of all files in the directory
file_list = os.listdir(folder_path)

# Filter the list to only include image files (e.g., .jpg, .png, .jpeg)
image_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed
image_paths = [os.path.join(folder_path, file) for file in file_list if any(file.lower().endswith(ext) for ext in image_extensions)]
  # Add the paths of the images you want to process

for image_path in image_paths:
    img = cv2.imread(image_path)
    
    img_height, img_width = img.shape[:2]  # Get the height and width of the image

    blob = cv2.dnn.blobFromImage(img, scalefactor=1/255, size=(640, 640), mean=[0, 0, 0], swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward()[0]

    # cx, cy, w, h, confidence, 80 class_scores
    # class_ids, confidences, boxes

    class_ids = []
    confidences = []
    boxes = []
    rows = detections.shape[0]

    x_scale = img_width / 640
    y_scale = img_height / 640

    for i in range(rows):
        row = detections[i]
        confidence = row[4]
        if confidence > 0.5:
            class_scores = row[5:]
            ind = np.argmax(class_scores)
            if class_scores[ind] > 0.5:
                class_ids.append(ind)
                confidences.append(confidence)
                cx, cy, w, h = row[:4]
                x1 = int((cx - w / 2) * x_scale)
                y1 = int((cy - h / 2) * y_scale)
                width = int(w * x_scale)
                height = int(h * y_scale)
                box = np.array([x1, y1, width, height])
                boxes.append(box)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.5)

    for i in indices:
        x1, y1, w, h = boxes[i]
        label = classes[class_ids[i]]
        conf = confidences[i]
        text = label + " {:.2f}".format(conf)

        # Crop the detection region
        crop_img = img[y1:y1+h, x1:x1+w]

        # Save cropped image with the respective class name
        save_path = f"{label}.jpg"
        cv2.imwrite(save_path, crop_img)

        cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (255, 0, 0), 2)
        cv2.putText(img, text, (x1, y1 - 2), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 255), 2)

    cv2.imwrite('my.jpg',img)

    cv2.waitKey(0)

    cv2.destroyAllWindows()


    import cv2
    import numpy as np

    # Load an image
    image = cv2.imread('b.jpg')

    # Create a sharpening kernel
    sharpening_kernel = np.array([[-1, -1, -1],
                                [-1,  9, -1],
                                [-1, -1, -1]])

    # Apply the convolution to sharpen the image
    sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)

    # Display the original and sharpened images

    cv2.imwrite('b.jpg', sharpened_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    import io
    import os

    from google.cloud import vision
    from google.cloud.vision_v1 import types

    # Set your environment variable for your GCP credentials
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'circular-genius-402810-13afeaaa2b14.json'

    # Initialize the Vision API client
    client = vision.ImageAnnotatorClient()

    # Load the image
    with io.open('b.jpg', 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    # Create the text detection request
    request = types.AnnotateImageRequest(
        image=image,
        features=[types.Feature(type='TEXT_DETECTION')]
    )

    # Send the request and get the response
    response = client.annotate_image(request)


    import json
    from google.protobuf.json_format import MessageToDict
    response_dict = MessageToDict(response._pb)

    # Access the textAnnotations
    text_annotations = response_dict.get('textAnnotations', [])

    # Function to calculate the center point of a bounding box
    def calculate_center(vertices):
        x_coordinates = [vertex['x'] for vertex in vertices]
        y_coordinates = [vertex['y'] for vertex in vertices]
        center_x = sum(x_coordinates) / len(vertices)
        center_y = sum(y_coordinates) / len(vertices)
        return center_x, center_y

    # Calculate center points for all bounding boxes along with descriptions
    center_points_with_descriptions = []
    for annotation in text_annotations[1:]:
        description = annotation.get('description', 'N/A')  # Use 'N/A' if description is not present
        vertices = annotation['boundingPoly']['vertices']
        center_x, center_y = calculate_center(vertices)
        center_points_with_descriptions.append({'description': description, 'x': center_x, 'y': center_y})



    data = center_points_with_descriptions
    df = pd.DataFrame(data)

    # Define the target number of rows in column 0
    target_rows_in_col0 = 6
    y_threshold = 40  # Start with the initial threshold

    while True:
        # Calculate the bins for x based on the number of columns
        x_bins = np.linspace(df['x'].min(), df['x'].max(), target_rows_in_col0 + 1)

        # Assign each point to a cell in the table
        df['column'] = np.digitize(df['x'], x_bins) - 1

        # Create 'y_bins' using the updated 'y_threshold'
        y_bins = np.arange(df['y'].min(), df['y'].max() + y_threshold, y_threshold)

        # Assign each point to a row in the table
        df['row'] = np.digitize(df['y'], y_bins) - 1

        # Calculate the number of unique rows in column 0
        num_unique_rows_in_col0 = len(df[df['column'] == 0]['row'].unique())
    
        
        # Calculate the number of rows dynamically based on the 'row' column
        num_rows = df['row'].max() + 1

        # Create a table with the specified number of rows and columns
        table = pd.DataFrame('', index=np.arange(num_rows), columns=np.arange(target_rows_in_col0))

        # Fill the table with the descriptions from the data
        for i, row in df.iterrows():
            table.at[row['row'], row['column']] = row['description']

        # If the number of unique rows in column 0 matches the target, exit the loop
        if table.shape[0] == 9:
            break

        # Increase the y_threshold for the next iteration
        y_threshold += 10  # You can adjust the step as needed

    # Calculate the number of rows dynamically based on the 'row' column
    num_rows = df['row'].max() + 1

    # Create a table with the specified number of rows and columns
    table = pd.DataFrame('', index=np.arange(num_rows), columns=np.arange(target_rows_in_col0))

    # Fill the table with the descriptions from the data
    for i, row in df.iterrows():
        table.at[row['row'], row['column']] = row['description']

    # Print the resulting table
    print(table)
    table.columns = table.columns.astype(str)
    table = table.drop(table.index[0])
    df=table
    new_df = df.replace('', np.nan)


# change to your key 
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'circular-genius-402810-13afeaaa2b14.json'

    # Initialize the Vision API client
    client = vision.ImageAnnotatorClient()

    # Load the image
    with io.open('a.jpg', 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    # Create the text detection request
    request = types.AnnotateImageRequest(
        image=image,
        features=[types.Feature(type='TEXT_DETECTION')]
    )

    # Send the request and get the response
    response = client.annotate_image(request)
    text_annotations = response.text_annotations
    import re
    # Get the first text annotation (which contains the entire text)

    full_text = text_annotations[0].description

    digits = re.sub(r'\D', '', full_text)


    # Load the existing DataFrame from the CSV file
    main_data = pd.read_csv('v.csv')

    # Your list containing question numbers and marks
    data_list = [digits] + list(new_df['1']) + list(new_df['3']) + list(new_df['5'])

    # Create a new row as a DataFrame with the same column names as the main_data
    new_row = pd.DataFrame([data_list], columns=main_data.columns)

    # Concatenate the new row to the existing DataFrame
    existing_df = pd.concat([main_data, new_row], ignore_index=True)
    existing_df=existing_df.fillna(0)

    # Save the updated DataFrame to the CSV file without creating new columns
    existing_df.to_csv('v.csv', index=False)


      0         1    2         3        4         5    6
0   No.  Obtained  No.  Obtained    3.000  Obtained  NaN
1     1              9                 17        IN  NaN
2     2         0   10                 15            NaN
3     0             11         1       19        22    /
4     4    3uiken   12                  1            NaN
5     5             13         !        d            NaN
6     6             14                 22            NaN
7     7             15                 23            NaN
8  Wazo         e  dyd       udz  24/1200    95asit  NaN
     0         1    2         3    4         5      6
0  No.  Obtained  No.  Obtained  No.  Obtained  Marks
1    1         o    9             17              NaN
2    2         2   10             18              NaN
3    3         4   11             19              NaN
4    4         5   12             20              NaN
5    5         .   13             21              NaN
6    6         1   14             22              Na

In [120]:
import cv2
import numpy as np

# Load an image

import io
import os

from google.cloud import vision
from google.cloud.vision_v1 import types

# Set your environment variable for your GCP credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'circular-genius-402810-13afeaaa2b14.json'

# Initialize the Vision API client
client = vision.ImageAnnotatorClient()

# Load the image
with io.open('a.jpg', 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Create the text detection request
request = types.AnnotateImageRequest(
    image=image,
    features=[types.Feature(type='TEXT_DETECTION')]
)

# Send the request and get the response
response = client.annotate_image(request)
text_annotations = response.text_annotations
import re
# Get the first text annotation (which contains the entire text)

full_text = text_annotations[0].description

full_text = 'Enrolment No.:___220479'
digits = re.sub(r'\D', '', full_text)

digits

'220479'

In [74]:
import cv2
import numpy as np

# Load an image
image = cv2.imread('bc.jpg')

# Create a sharpening kernel
sharpening_kernel = np.array([[-1, -1, -1],
                              [-1,  9, -1],
                              [-1, -1, -1]])

# Apply the convolution to sharpen the image
sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)

# Display the original and sharpened images

cv2.imwrite('bc.jpg', sharpened_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
import io
import os

from google.cloud import vision
from google.cloud.vision_v1 import types

# Set your environment variable for your GCP credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'circular-genius-402810-13afeaaa2b14.json'

# Initialize the Vision API client
client = vision.ImageAnnotatorClient()

# Load the image
with io.open('bc.jpg', 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Create the text detection request
request = types.AnnotateImageRequest(
    image=image,
    features=[types.Feature(type='TEXT_DETECTION')]
)

# Send the request and get the response
response = client.annotate_image(request)
g=[]
for text_annotation in response.text_annotations:
    print('Text:', text_annotation.description)
    g.append(text_annotation.description)



Text: Question Marks Question Diarks Question Marks
No. Obtained
No.
Hobtained
No. Obtained
4
19
10
11
12
13
14
11
on
5
00
16
21st
18
19
20
21
22
23
24
Text: Question
Text: Marks
Text: Question
Text: Diarks
Text: Question
Text: Marks
Text: No.
Text: Obtained
Text: No.
Text: Hobtained
Text: No.
Text: Obtained
Text: 4
Text: 19
Text: 10
Text: 11
Text: 12
Text: 13
Text: 14
Text: 11
Text: on
Text: 5
Text: 00
Text: 16
Text: 21st
Text: 18
Text: 19
Text: 20
Text: 21
Text: 22
Text: 23
Text: 24


In [75]:
import json
from google.protobuf.json_format import MessageToDict
response_dict = MessageToDict(response._pb)

# Access the textAnnotations
text_annotations = response_dict.get('textAnnotations', [])

# Function to calculate the center point of a bounding box
def calculate_center(vertices):
    x_coordinates = [vertex['x'] for vertex in vertices]
    y_coordinates = [vertex['y'] for vertex in vertices]
    center_x = sum(x_coordinates) / len(vertices)
    center_y = sum(y_coordinates) / len(vertices)
    return center_x, center_y

# Calculate center points for all bounding boxes along with descriptions
center_points_with_descriptions = []
for annotation in text_annotations[1:]:
    description = annotation.get('description', 'N/A')  # Use 'N/A' if description is not present
    vertices = annotation['boundingPoly']['vertices']
    center_x, center_y = calculate_center(vertices)
    center_points_with_descriptions.append({'description': description, 'x': center_x, 'y': center_y})

# Print the center points with descriptions
print(json.dumps(center_points_with_descriptions, indent=2))
import pandas as pd
import numpy as np

data = center_points_with_descriptions
df = pd.DataFrame(data)

# Define a threshold for y values to consider them as belonging to the same row
y_threshold = 50

# Define the number of columns
num_cols = 6

# Calculate the bins for x based on the number of columns
x_bins = np.linspace(df['x'].min(), df['x'].max(), num_cols + 1)

# Assign each point to a cell in the table
df['column'] = np.digitize(df['x'], x_bins) - 1

# Create 'y_bins' using 'y_threshold'
y_bins = np.arange(df['y'].min(), df['y'].max() + y_threshold, y_threshold)

# Assign each point to a row in the table
df['row'] = np.digitize(df['y'], y_bins) - 1

# Calculate the number of rows dynamically based on the 'row' column
num_rows = df['row'].max() + 1

# Create a table with the specified number of rows and columns
table = pd.DataFrame('', index=np.arange(num_rows), columns=np.arange(num_cols))

# Fill the table with the descriptions from the data
for i, row in df.iterrows():
    table.at[row['row'], row['column']] = row['description']

# Print the resulting table
print(table)

[
  {
    "description": "Question",
    "x": 77.0,
    "y": 42.5
  },
  {
    "description": "Marks",
    "x": 230.5,
    "y": 46.0
  },
  {
    "description": "Question",
    "x": 384.0,
    "y": 46.0
  },
  {
    "description": "Diarks",
    "x": 534.0,
    "y": 46.0
  },
  {
    "description": "Question",
    "x": 686.5,
    "y": 46.0
  },
  {
    "description": "Marks",
    "x": 836.0,
    "y": 46.0
  },
  {
    "description": "No.",
    "x": 76.0,
    "y": 81.0
  },
  {
    "description": "Obtained",
    "x": 232.0,
    "y": 83.5
  },
  {
    "description": "No.",
    "x": 384.0,
    "y": 86.0
  },
  {
    "description": "Hobtained",
    "x": 528.5,
    "y": 85.0
  },
  {
    "description": "No.",
    "x": 687.0,
    "y": 87.5
  },
  {
    "description": "Obtained",
    "x": 839.5,
    "y": 87.5
  },
  {
    "description": "4",
    "x": 234.0,
    "y": 144.5
  },
  {
    "description": "19",
    "x": 380.0,
    "y": 153.0
  },
  {
    "description": "10",
    "x": 383.0,
    "y":

In [76]:
import pandas as pd
import numpy as np

data = center_points_with_descriptions
df = pd.DataFrame(data)

# Define a threshold for y values to consider them as belonging to the same row
y_threshold = 50

# Define the number of columns
num_cols = 6

# Calculate the bins for x based on the number of columns
x_bins = np.linspace(df['x'].min(), df['x'].max(), num_cols + 1)

# Assign each point to a cell in the table
df['column'] = np.digitize(df['x'], x_bins) - 1

# Create 'y_bins' using 'y_threshold'
y_bins = np.arange(df['y'].min(), df['y'].max() + y_threshold, y_threshold)

# Assign each point to a row in the table
df['row'] = np.digitize(df['y'], y_bins) - 1

# Calculate the number of rows dynamically based on the 'row' column
num_rows = df['row'].max() + 1

# Create a table with the specified number of rows and columns
table = pd.DataFrame('', index=np.arange(num_rows), columns=np.arange(num_cols))

# Fill the table with the descriptions from the data
for i, row in df.iterrows():
    table.at[row['row'], row['column']] = row['description']

# Print the resulting table
print(table)

      0         1     2          3    4      5         6
0   No.  Obtained   No.  Hobtained  No.  Marks  Obtained
1                                                    NaN
2    11         4    19                              NaN
3                  21st              18              NaN
4    on              11              19              NaN
5                    12              20              NaN
6                                                    NaN
7     5              13              21              NaN
8                    14              22              NaN
9                                    23              NaN
10   00              16              24              NaN


In [12]:
# red image table 
import numpy as np
import cv2

image_path = 'b.jpg'
image = cv2.imread(image_path)


    # Get the height and width of the image
image_height, image_width, _ = image.shape
 # Height of the blank image
blank_image = np.zeros((image_height, image_width, 3), np.uint8)

# Draw circles and text on the blank image
for point in center_points_with_descriptions:
    x, y = int(point['x']), int(point['y'])
    description = point['description']

    # Draw text
    cv2.putText(blank_image, description, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)  # Red text

# Define the number of columns and draw vertical lines
num_columns = 6  # Adjust this to match the number of columns in your data
column_width = image_width // num_columns
for i in range(1, num_columns):
    x = i * column_width
    cv2.line(blank_image, (x, 0), (x, image_height), (0, 0, 255), 2)  # Draw red vertical lines

# Display or save the image
cv2.imshow('Annotated Image', blank_image)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()

# Save the image to a file
cv2.imwrite('annotated_image.jpg', blank_image)


True

In [10]:
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Example path, replace with your Tesseract path

# Read the image
img = cv2.imread('annotated_image.jpg')


# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding to invert the image
binary_thresh = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                      cv2.THRESH_BINARY, 15, -2)

# Copy the binary image
vertical = binary_thresh.copy()

# Define the structure size for the vertical kernel
vertical_size = vertical.shape[0] // 30

# Create a vertical kernel
vertical_structure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, vertical_size))

# Apply morphological operations
vertical = cv2.erode(vertical, vertical_structure)
vertical = cv2.dilate(vertical, vertical_structure)

# Use pytesseract to extract text
config = ('-l eng --oem 1 --psm 3')
text = pytesseract.image_to_string(vertical)

# Post-process the extracted text
rows = text.split('\n')
table = [row.split() for row in rows]

# Print the extracted table data
for row in table:
    print(row)

[]


In [12]:
import cv2
import pytesseract
import pandas as pd

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Example path, replace with your Tesseract path

# Read the image
image = cv2.imread('annotated_image.jpg')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# Find the contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on their area (you may need to adjust the value)
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 50]

# Create an empty DataFrame
df = pd.DataFrame()

for i, contour in enumerate(contours):
    # Get the bounding rectangle for the contour
    x, y, w, h = cv2.boundingRect(contour)

    # Slice the image using the bounding rectangle
    roi = gray[y:y+h, x:x+w]

    # Extract text from the region of interest
    text = pytesseract.image_to_string(roi)

    # Split the text into lines and values
    lines = text.splitlines()
    values = [line.split() for line in lines if line.strip()]

    # If values are less than expected, append None values to compensate
    num_columns = len(values[0])  # Assuming the number of columns is the same for all rows
    for value in values:
        if len(value) < num_columns:
            value += [None] * (num_columns - len(value))

    # Append the values to the DataFrame
    df_temp = pd.DataFrame(values, columns=["Column1", "Column2"])
    df = pd.concat([df, df_temp], ignore_index=True)

# Specify the columns to keep (excluding the "name" column)
columns_to_keep = [col for col in df.columns if col != 'name']

# Create a new DataFrame with the desired columns
df_cleaned = df[columns_to_keep]

# Display the cleaned DataFrame
print(df_cleaned)


     Column1    Column2
0   Question      Marks
1        No.  Obtained}
2          1         25
3          2          3
4          3         2.
5          4          5
6          5          5
7          6         35
8          7         45
9        No,       None
10        10       None
11        Ww       None
12        12       None
13        13       None
14        14       None
15        15       None
16        16       None
17     Morks       None
18    Obtait       None
19  Question       None
20       No.       None
21        WW       None
22        18       None
23        19       None
24         2       None
25        24       None
26     Morks       None
27  Obtained       None


In [4]:
%cd yolov5

c:\freelancer\student ocr\yolov5


True